In [ ]:
### label에 조사 있는지 파악 ###
from konlpy.tag import Twitter
 
twitter = Twitter()
twitter.pos('나는 밥을 먹는다')

count = 0
for i in range(len(dataset)):
  for word, tag in twitter.pos(dataset[i]['answers'][0]['text']):
    if tag == 'Josa':
      count += 1

print(f'조사가 포함된 answers 수: {count}')
print(f'전체 answer에서 조사 비율: {count/len(dataset)*100}%')

In [ ]:
### 발표용 KoEDA 예시 ###
with open(path + 'train.json', 'r', encoding='utf-8') as fd:
  data = json.load(fd)

from koeda import EDA

eda = EDA(morpheme_analyzer="Okt", alpha_sr=0.1, alpha_ri=0.1, alpha_rs=0.1, prob_rd=0.1)

data['data'].append(data['data'][0])
context = data['data'][0]['paragraphs'][0]['context']
answer = data['data'][0]['paragraphs'][0]['qas'][0]['answers'][0]['text']
new_context = eda(context, p=(0.1, 0.1, 0.1, 0.1), repetition=1)
answer = data['data'][0]['paragraphs'][0]['qas'][0]['answers'][0]['text']

print(context)
print(new_context)
print(answer)

answer in new_context

In [ ]:
# 최초 한 번만 실행
# KoEDA data 생성

with open(path + 'train.json', 'r', encoding='utf-8') as fd:
  data = json.load(fd)

import copy
from tqdm import tqdm
from koeda import EDA

eda = EDA(morpheme_analyzer="Okt", alpha_sr=0.1, alpha_ri=0.1, alpha_rs=0.1, prob_rd=0.1)

length = len(data['data']) 
print(f'원본 data 길이: {length}')
new_data = copy.deepcopy(data)

for idx in tqdm(range(length)):
  context = data['data'][idx]['paragraphs'][0]['context']
  answer = data['data'][idx]['paragraphs'][0]['qas'][0]['answers'][0]['text']

  while True:
    new_context = eda(context, p=(0.7, 0.7, 0.7, 0.7), repetition=1)
    if (answer in new_context) and (context != new_context):
      break
  
  new_data['data'][idx]['paragraphs'][0]['context'] = new_context

print('\n', f'augmentation data 길이: ', len(new_data['data']), '\n')
print(data['data'][0])
print(new_data['data'][0])

In [ ]:
# new_data의 answer_start 변경
for idx in range(len(new_data['data'])):
  context = new_data['data'][idx]['paragraphs'][0]['context']
  answer = new_data['data'][idx]['paragraphs'][0]['qas'][0]['answers'][0]['text']
  answer_start = context.find(answer)
  new_data['data'][idx]['paragraphs'][0]['qas'][0]['answers'][0]['answer_start'] = answer_start

print(data['data'][0]['paragraphs'][0]['context'])
print(data['data'][0]['paragraphs'][0]['qas'][0]['answers'][0]['answer_start'])
print(new_data['data'][0]['paragraphs'][0]['context'])
print(new_data['data'][0]['paragraphs'][0]['qas'][0]['answers'][0]['answer_start'])

In [ ]:
# 기존 data와 new_data 병합
length = len(new_data['data'])
for idx in range(length):
  data['data'].append(new_data['data'][idx])

print(len(data['data']))
print(data['data'][9789])

# 병합 파일 저장
with open(path + 'KoEDA_train.json', 'w') as new:
  json.dump(data, new)